## Testing Model for MIDI file generator

In [27]:
import pandas as pd
import random
import pretty_midi
from basic_pitch import ICASSP_2022_MODEL_PATH, inference
from pedalboard import Pedalboard, Reverb, Chorus, Distortion, Delay, Phaser, Compressor, Gain, Clipping
from pedalboard.io import AudioFile
from midi2audio import FluidSynth 
from pydub import AudioSegment
import tempfile
import concurrent.futures
import os
import shutil
from pathlib import Path

In [3]:
def create_individual(effects, effect_structure):
    n_effects_chosen = random.randint(1, len(effects))
    selected_effects = random.sample(effects, n_effects_chosen)
    
    individ = {}
    for effect in selected_effects:
        if effect in effect_structure:
            structure = effect_structure[effect]
            individ[effect] = {
                param: round(random.uniform(range_[0], range_[1]), 2) 
                for param, (_, range_) in structure.items()
            }
    return individ

In [4]:
def generate_random_instrument_midi(filename, program, min_duration, max_duration, min_note, max_note):
    # Create a PrettyMIDI object
    midi_data = pretty_midi.PrettyMIDI()
    
    # Create an Instrument instance for a distortion instrument
    instrument = pretty_midi.Instrument(program=program)
    
    # Determine the length of the MIDI in seconds
    duration = random.randint(min_duration, max_duration)
    min_note_number = pretty_midi.note_name_to_number(min_note)
    max_note_number = pretty_midi.note_name_to_number(max_note)
    
    # Generate random notes
    current_time = 0.0
    while current_time < duration:
        note_number = random.randint(min_note_number, max_note_number)
        
        note_duration = random.uniform(0.1, 1.0)
        
        # Ensure that the note ends before the total duration
        note_end_time = min(current_time + note_duration, duration)
        
        # Create a Note instance and add it to the instrument instrument
        note = pretty_midi.Note(
            velocity=random.randint(60, 127),  # Random velocity
            pitch=note_number,
            start=current_time,
            end=note_end_time
        )
        instrument.notes.append(note)
        
        # Update the current time
        current_time = note_end_time
    
    # Add the instrument instrument to the PrettyMIDI object
    midi_data.instruments.append(instrument)
    
    # Write out the MIDI data to a file
    midi_data.write(filename)
    
    return midi_data


In [5]:
def create_effected_audio(board, file_path, output_file):
    with AudioFile(file_path) as f:
        with AudioFile(output_file, 'w', f.samplerate, f.num_channels) as o:
            while f.tell() < f.frames:
                chunk = f.read(f.samplerate)
                effected = board(chunk, f.samplerate, reset=False)
                o.write(effected)

In [6]:
def midi_to_mp3(midi_file, mp3_file, soundfont):
    #convert MIDI to WAV using FluidSynth
    fs = FluidSynth(soundfont)
    wav_file = midi_file.replace('.midi', '.wav').replace('.mid', '.wav')
    fs.midi_to_audio(midi_file, wav_file)

    #convert WAV to MP3 using pydub
    sound = AudioSegment.from_wav(wav_file)
    sound.export(mp3_file, format="mp3")
    
def mp3_to_midi(audio_path):
    _, midi_data, __ = inference.predict(
        audio_path,    
        model_or_model_path = ICASSP_2022_MODEL_PATH, 
        onset_threshold = 0.6, #note segmentation 1) how easily a note should be split into two. (split notes <- ..0.5.. -> merge notes)
        frame_threshold = 0.6, #model confidence threshold 2) the model confidence required to create a note. (more notes <- ..0.3.. -> fewer notes)
    )

    for instrument in midi_data.instruments:
        instrument.program = 30 #distortion guitar program
            
    return midi_data

In [15]:
def test_model_db_creation(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map):
    for _ in range(n_data):
        midi_data_original = generate_random_instrument_midi(midi_file, program, min_duration, max_duration, min_note, max_note)
        midi_to_mp3(midi_file, mp3_file, soundfont)
        individual = create_individual(effects, effect_structure)
        board = Pedalboard([])
        for effect_key, params in individual.items():
            effect_class = globals()[effects_map[effect_key]]
            board.append(effect_class(**params))
            
        effected_audio_name = "audios/effected_audio.mp3"
        create_effected_audio(board, mp3_file, effected_audio_name)
        new_midi_generated = mp3_to_midi(effected_audio_name)
        data["original_midi_data"].append(midi_data_original.instruments[0].notes)
        if new_midi_generated.instruments:
            data["generated_midi_data"].append(new_midi_generated.instruments[0].notes)
        else:
            data["generated_midi_data"].append(new_midi_generated.instruments)
    
    # Once all the data is collected, you can create the DataFrame
    df = pd.DataFrame(data)
    
    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"CSV file '{csv_filename}' created successfully.")

In [24]:
def process_data_generation(index, midi_file, mp3_file, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map):
    # Create temporary MIDI and MP3 files for each parallel process
    with tempfile.NamedTemporaryFile(suffix='.mid', delete=False) as temp_midi, tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as temp_mp3:
        temp_midi_name = temp_midi.name
        temp_mp3_name = temp_mp3.name
        
        # Copy the original midi and mp3 file to the temporary file locations
        shutil.copy(midi_file, temp_midi_name)
        shutil.copy(mp3_file, temp_mp3_name)
        
        # Dictionary to hold the data for this individual run
        result_data = {
            "original_midi_data": [],
            "generated_midi_data": [],
        }

        # Generate the MIDI data and apply effects
        midi_data_original = generate_random_instrument_midi(temp_midi_name, program, min_duration, max_duration, min_note, max_note)
        midi_to_mp3(temp_midi_name, temp_mp3_name, soundfont)
        individual = create_individual(effects, effect_structure)
        board = Pedalboard([])
        for effect_key, params in individual.items():
            effect_class = globals()[effects_map[effect_key]]
            board.append(effect_class(**params))
        
        # Apply effects and create the effected audio
        effected_audio_name = tempfile.mktemp(suffix='.mp3')  # Temporary file for effected audio
        create_effected_audio(board, temp_mp3_name, effected_audio_name)
        new_midi_generated = mp3_to_midi(effected_audio_name)

        result_data["original_midi_data"].append(midi_data_original.instruments[0].notes)
        if new_midi_generated.instruments:
            result_data["generated_midi_data"].append(new_midi_generated.instruments[0].notes)
        else:
            result_data["generated_midi_data"].append(new_midi_generated.instruments)

        # Clean up temporary files
        Path(temp_midi_name).unlink(missing_ok=True)
        Path(temp_mp3_name).unlink(missing_ok=True)
        Path(effected_audio_name).unlink(missing_ok=True)

        return result_data

In [29]:
def test_model_db_creation_parallel(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map):
    n_workers = os.cpu_count() - 1
    # Use ProcessPoolExecutor for parallel processing
    with concurrent.futures.ProcessPoolExecutor(max_workers=n_workers) as executor:
        futures = [
            executor.submit(process_data_generation, i, midi_file, mp3_file, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map)
            for i in range(n_data)
        ]
        
        # Collect results as they complete
        for future in concurrent.futures.as_completed(futures):
            result_data = future.result()
            data["original_midi_data"].extend(result_data["original_midi_data"])
            data["generated_midi_data"].extend(result_data["generated_midi_data"])

    # Once all the data is collected, create the DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"CSV file '{csv_filename}' created successfully.")

In [30]:
data = {
    "original_midi_data": [],
    "generated_midi_data": [], 
}

n_data = 10000
midi_file = 'output.mid'
mp3_file = 'output.mp3'
csv_filename = "dataset_midis.csv"
soundfont = '../audio2midi2audio/FluidR3_GM.sf2'  # Path to your SoundFont file
program = 30
min_note = 'E2'
max_note = 'E6' # Generate a random note between MIDI note 40 (E2) and 88 (E6)
min_duration = 3
max_duration = 20
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (0.0, 100.0)), },# Chorus
    1: { "delay_seconds": ('float', (0.0, 10.0)), },# Delay
    2: { "drive_db": ('float', (0.0, 50.0)), },# Distortion
    3: { "gain_db": ('float', (-50.0, 50.0)) },# Gain
    4: { "depth": ('float', (0.0, 1.0)), },# Phaser
    5: { "wet_level": ('float', (0.0, 1.0)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [31]:
test_model_db_creation_parallel(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [21]:
test_model_db_creation(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map)

Predicting MIDI for audios/effected_audio.mp3...
Predicting MIDI for audios/effected_audio.mp3...
Predicting MIDI for audios/effected_audio.mp3...
Predicting MIDI for audios/effected_audio.mp3...
Predicting MIDI for audios/effected_audio.mp3...
CSV file 'dataset_midis.csv' created successfully.


In [19]:
midi_data = mp3_to_midi('audios/effected_audio.mp3')
# Using __dict__ to see instance attributes
print(midi_data.__dict__)

# Accessing some specific properties
print("Tempo changes:", midi_data.get_tempo_changes())
print("Time signature changes:", midi_data.time_signature_changes)
print("Instruments:", midi_data.instruments)
print(midi_data)

Predicting MIDI for audios/effected_audio.mp3...
{'resolution': 220, '_tick_scales': [(0, 0.0022727272727272726)], '_PrettyMIDI__tick_to_time': [0], 'instruments': [], 'key_signature_changes': [], 'time_signature_changes': [], 'lyrics': [], 'text_events': []}
Tempo changes: (array([0.]), array([120.]))
Time signature changes: []
Instruments: []
